# Импорт библиотек

In [205]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import nltk
import faiss
import string

# Считываю датасет

In [206]:
df = pd.read_parquet('train-00000-of-00001.parquet')
all_texts = list(set(df['question_1'].to_list() + df['question_2'].to_list())) # Всего 4567 уникальных вопроса
question_1 = list(set(df['question_1'].to_list()))
df.head(10)

,dr_id,question_1,question_2,label
0,1,After how many hour from drinking an antibioti...,I have a party tonight and I took my last dose...,1
1,1,After how many hour from drinking an antibioti...,I vomited this morning and I am not sure if it...,0
2,1,Am I over weight (192.9) for my age (39)?,I am a 39 y/o male currently weighing about 19...,1
3,1,Am I over weight (192.9) for my age (39)?,What diet is good for losing weight? Keto or v...,0
4,1,Aspirin allergy - is it worth getting a bracelet?,How much Aspirin can I take for my headache wi...,0
5,1,Aspirin allergy - is it worth getting a bracelet?,My friend told me about this bracelet for Aspi...,1
6,1,"At a doctor's visit, I hit my head against a b...",Is it okay if I use the same syringe for my in...,0
7,1,"At a doctor's visit, I hit my head against a b...","Today morning, I had an appointment with the d...",1
8,1,Been on antibiotics 4 5wks top high tooth dent...,"I am unable to get tooth extraction, my dentis...",1
9,1,Been on antibiotics 4 5wks top high tooth dent...,Why do I have black teeth? Should I take antib...,0


# Препроцессинг
Токенизация

In [207]:
def tokenize(text):
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

Стемминг

In [208]:
analyzer = CountVectorizer().build_analyzer()
stemmer = nltk.stem.PorterStemmer()
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

Векторизация

In [209]:
def preproc_df(text):
    vectorizer = CountVectorizer(analyzer = stemmed_words, 
                                lowercase = True, 
                                tokenizer = tokenize, 
                                preprocessor = None, 
                                stop_words = 'english',
                                ngram_range=(1, 3), 
                                max_features = 3500)
    bow_cv = vectorizer.fit(all_texts)
    bow_cv = vectorizer.transform(text)
    tfidf = TfidfTransformer().fit_transform(bow_cv)
    tfidf = tfidf.toarray()
    return tfidf

q_12 = preproc_df(all_texts)
print(q_12.shape)

c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:541: UserWarning: The parameter 'ngram_range' will not be used since 'analyzer' is callable'
  warnings.warn(
c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:547: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:560: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


(4567, 3500)


In [210]:
# dims = q_12.shape[1]
# n_cells = 5
# quantizer = faiss.IndexFlatL2(dims)
# idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
# idx_l2.train(np.ascontiguousarray(q_12).astype('float32'))
# idx_l2.add(np.ascontiguousarray(q_12).astype('float32'))
# candidate_number = 6
# D, I = idx_l2.search(np.ascontiguousarray(q_12).astype('float32'), candidate_number)
# print(I, D)

# Считаю соседей

In [211]:
index = faiss.IndexFlatL2(q_12.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(q_12)
print(index.ntotal)

topn = 6
D, I = index.search(q_12, topn)  # Возвращает результат: Distances, Indices
print(I)
print(D)

0
4567
[[   0 2786 2943 1135 3743 2359]
 [   1 1494 3962 1995  634  391]
 [   2   90 3583 2311 1724 2088]
 ...
 [4564  154 1404 4422 2890 1287]
 [4565  207  989 1833 3596 4398]
 [4566 1234 1956 3383 4167 1215]]
[[1.1920929e-07 1.2405272e+00 1.2738986e+00 1.2981002e+00 1.3534216e+00
  1.3571531e+00]
 [0.0000000e+00 8.5487652e-01 1.4408283e+00 1.5069506e+00 1.5079033e+00
  1.5487657e+00]
 [0.0000000e+00 9.9774647e-01 1.1876482e+00 1.1914618e+00 1.2179356e+00
  1.2654352e+00]
 ...
 [1.1920929e-07 4.0009582e-01 9.4201314e-01 1.3358542e+00 1.4795856e+00
  1.4858441e+00]
 [0.0000000e+00 1.1145983e+00 1.2115252e+00 1.2515256e+00 1.2633417e+00
  1.3046100e+00]
 [1.1920929e-07 1.2079360e+00 1.2600505e+00 1.5365478e+00 1.5526901e+00
  1.5636616e+00]]


Привожу датафреймы в массивы для подсчета метрик

In [212]:
result = []
for ind in I:
    row = []
    for i in ind:
        row.append(all_texts[i])
    result.append(row)

    
df_dict = df.to_dict()
df_mass = []
df = df[df['label'] == 1]
for i in range(len(df)):
    df_mass.append([df_dict['question_1'][i], df_dict['question_2'][i]])
print(len(df_mass), df_mass)

1524 [['After how many hour from drinking an antibiotic can I drink alcohol?', 'I have a party tonight and I took my last dose of Azithromycin this morning. Can I have a few drinks?'], ['After how many hour from drinking an antibiotic can I drink alcohol?', 'I vomited this morning and I am not sure if it is the side effect of my antibiotic or the alcohol I took last night...'], ['Am I over weight (192.9) for my age (39)?', 'I am a 39 y/o male currently weighing about 193 lbs. Do you think I am overweight?'], ['Am I over weight (192.9) for my age (39)?', 'What diet is good for losing weight? Keto or vegan?'], ['Aspirin allergy - is it worth getting a bracelet?', 'How much Aspirin can I take for my headache without causing any side effects?'], ['Aspirin allergy - is it worth getting a bracelet?', 'My friend told me about this bracelet for Aspirin allergy. Does it work the way they say it does?'], ["At a doctor's visit, I hit my head against a box on the wall containing hazardous material

# Accuracy @ 5

In [213]:
counter = 0
for row_df in df_mass:
    for row_res in result:
        if row_df[0] == row_res[0] and row_df[1] in row_res[1:]:
            counter += 1
            break
print(counter, len(df_mass), counter / len(df_mass))

1122 1524 0.7362204724409449


Проверяю скрипт на кастомном вопросе

In [218]:
a = ['Can I drink alcohol every day?']
a = preproc_df(a)
topn = 5
D, I = index.search(a, topn)

for el in I[0]:
    print(all_texts[el])

c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:541: UserWarning: The parameter 'ngram_range' will not be used since 'analyzer' is callable'
  warnings.warn(
c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:547: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
c:\Users\Артем Васильев\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:560: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


After how many hour from drinking an antibiotic can I drink alcohol?
I smoke and drink alcohol, is my sperm healthy? Can it last 7 days inside the uterus?
I have not drank alcohol for a year and 8 months now. Can recently drinking 0.06 pack a day affect my liver?
Effects of drinking alcohol on the body fat?
Can I drink alcohol after recovering from bird flu?


In [215]:
# from scipy.spatial.distance import pdist

# arr0 = pdist(q_12)  

# print("Arithmetic Mean is :", arr0) 